In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/gloriaye/Downloads/DesktopOrganizer/Main/dsc180b/sepsis_comorbidity_df.csv")

In [3]:
df

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3,hadm_id
0,18421337,30000484,2136-01-14 21:00:00,2136-01-14 18:10:00,2136-01-14 18:10:00,2136-01-14 19:00:00,3,0,0,0,0,3,0,True,22413411.0
1,12207593,30000646,2194-04-29 07:00:00,2194-04-29 01:00:00,2194-04-29 01:00:00,2194-04-29 11:00:00,3,2,0,0,1,0,0,True,22795209.0
2,15726459,30000831,2140-04-18 18:00:00,2140-04-18 05:11:00,2140-04-18 05:11:00,2140-04-17 22:00:00,5,0,0,0,0,3,2,True,NaN
3,16513856,30001446,2186-04-12 04:00:00,2186-04-11 08:20:00,2186-04-11 08:20:00,2186-04-12 04:00:00,8,0,3,3,0,0,2,True,24463832.0
4,10656173,30001555,2177-09-27 16:00:00,2177-09-27 07:21:00,2177-09-27 07:21:00,2177-09-27 12:00:00,8,0,3,4,0,1,0,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41290,15954569,39998871,2180-02-24 10:00:00,2180-02-24 04:05:00,2180-02-24 04:05:00,2180-02-24 08:00:00,3,0,0,0,3,0,0,True,25851401.0
41291,19425920,39999168,2190-04-11 21:00:00,2190-04-11 21:57:00,2190-04-11 21:00:00,2190-04-11 20:00:00,2,0,0,0,1,1,0,True,NaN
41292,15669140,39999172,2185-02-17 20:00:00,2185-02-18 12:45:00,2185-02-17 20:00:00,2185-02-17 18:00:00,2,0,0,0,1,0,1,True,29818488.0
41293,13651601,39999230,2147-09-01 02:00:00,2147-08-31 20:59:00,2147-08-31 20:59:00,2147-08-31 23:00:00,2,0,0,0,1,0,1,True,22584645.0


In [9]:
from sqlalchemy import create_engine, text

# Connect to database
engine = create_engine("postgresql+psycopg2://gloriaye@localhost:5532/mimiciv")

# 1. List all materialized views in mimiciv_derived schema
print("=" * 60)
print("Materialized Views in mimiciv_derived schema:")
print("=" * 60)
matviews_query = """
SELECT 
    schemaname, 
    matviewname,
    pg_size_pretty(pg_total_relation_size(schemaname||'.'||matviewname)) as size
FROM pg_matviews 
WHERE schemaname = 'mimiciv_derived'
ORDER BY matviewname;
"""
matviews_df = pd.read_sql(matviews_query, engine)
print(matviews_df.to_string(index=False))

Materialized Views in mimiciv_derived schema:
     schemaname      matviewname    size
mimiciv_derived mv_icd9_patients 5832 kB


In [14]:
# Create all materialized views (run this first if views don't exist)
from pathlib import Path

matview_scripts = [
    'matviews/mv_icd9_patients.sql',
    'matviews/mv_first_icu_stay.sql',
    'matviews/mv_icd9_icu_cohort_data.sql',
    'matviews/mv_management_view_data.sql'
]

print("Creating materialized views...")
print("=" * 60)

for script_path in matview_scripts:
    script_file = Path(script_path)
    if script_file.exists():
        print(f"\nRunning: {script_path}")
        with open(script_file, 'r') as f:
            sql_content = f.read()
        try:
            with engine.begin() as conn:
                conn.execute(text(sql_content))
            print(f"✓ Successfully created: {script_file.name}")
        except Exception as e:
            print(f"✗ Error: {str(e)[:200]}...")
    else:
        print(f"✗ File not found: {script_path}")

print("\n" + "=" * 60)
print("Done!")

Creating materialized views...

Running: matviews/mv_icd9_patients.sql
✓ Successfully created: mv_icd9_patients.sql

Running: matviews/mv_first_icu_stay.sql
✓ Successfully created: mv_first_icu_stay.sql

Running: matviews/mv_icd9_icu_cohort_data.sql
✓ Successfully created: mv_icd9_icu_cohort_data.sql

Running: matviews/mv_management_view_data.sql
✓ Successfully created: mv_management_view_data.sql

Done!


In [15]:
# 2. View data from each materialized view (with error handling)
matview_names = [
    'mv_icd9_patients',
    'mv_first_icu_stay',
    'mv_icd9_icu_cohort_data',
    'mv_management_view_data'
]

for mv_name in matview_names:
    print(f"\n{'='*60}")
    print(f"Viewing: mimiciv_derived.{mv_name}")
    print(f"{'='*60}")
    
    try:
        # Get row count
        count_query = f"SELECT COUNT(*) as row_count FROM mimiciv_derived.{mv_name};"
        count_df = pd.read_sql(count_query, engine)
        print(f"Total rows: {count_df['row_count'].iloc[0]:,}")
        
        # Get sample data
        sample_query = f"SELECT * FROM mimiciv_derived.{mv_name} LIMIT 5;"
        sample_df = pd.read_sql(sample_query, engine)
        print(f"\nFirst 5 rows:")
        display(sample_df)
        print(f"\nColumns: {list(sample_df.columns)}")
        print(f"Shape: {sample_df.shape}")
    except Exception as e:
        print(f"⚠ Materialized view does not exist yet!")
        print(f"   Error: {str(e)[:150]}...")
        print(f"   Run the previous cell to create all materialized views.")


Viewing: mimiciv_derived.mv_icd9_patients
Total rows: 100,386

First 5 rows:


,subject_id
0,18369563
1,15189519
2,14541250
3,17154054
4,17169170



Columns: ['subject_id']
Shape: (5, 1)

Viewing: mimiciv_derived.mv_first_icu_stay
Total rows: 65,366

First 5 rows:


,subject_id,hadm_id,stay_id
0,10000032,29079034,39553978
1,10000690,25860671,37081114
2,10000980,26913865,39765666
3,10001217,24597018,37067082
4,10001725,25563031,31205490



Columns: ['subject_id', 'hadm_id', 'stay_id']
Shape: (5, 3)

Viewing: mimiciv_derived.mv_icd9_icu_cohort_data
Total rows: 391,717

First 5 rows:


,subject_id,hadm_id,stay_id,seq_num,icd_code,icd_version,long_title,admittime,dischtime,hospital_expire_flag
0,10016810,20973395,35436337,10,V4577,9,"Acquired absence of organ, genital organs",2185-06-16 01:31:00,2185-06-21 15:55:00,0
1,10016810,20973395,35436337,9,V1082,9,Personal history of malignant melanoma of skin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
2,10016810,20973395,35436337,8,V5866,9,Long-term (current) use of aspirin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
3,10016810,20973395,35436337,7,3051,9,Tobacco use disorder,2185-06-16 01:31:00,2185-06-21 15:55:00,0
4,10016810,20973395,35436337,6,25000,9,Diabetes mellitus without mention of complicat...,2185-06-16 01:31:00,2185-06-21 15:55:00,0



Columns: ['subject_id', 'hadm_id', 'stay_id', 'seq_num', 'icd_code', 'icd_version', 'long_title', 'admittime', 'dischtime', 'hospital_expire_flag']
Shape: (5, 10)

Viewing: mimiciv_derived.mv_management_view_data
Total rows: 35,802

First 5 rows:


,subject_id,hadm_id,stay_id,gender,admission_age,race,icu_intime,icu_outtime,los_icu,hospital_expire_flag
0,10000032,29079034,39553978,F,52.559969,WHITE,2180-07-23 14:00:00,2180-07-23 23:50:47,0.41,0
1,10000690,25860671,37081114,F,86.837120,WHITE,2150-11-02 19:37:00,2150-11-06 17:03:17,3.89,0
2,10001217,24597018,37067082,F,55.881486,WHITE,2157-11-20 19:18:02,2157-11-21 22:08:00,1.12,0
3,10001217,27703517,34592300,F,55.962942,WHITE,2157-12-19 15:42:24,2157-12-20 14:27:41,0.95,0
4,10001725,25563031,31205490,F,46.275517,WHITE,2110-04-11 15:52:22,2110-04-12 23:59:56,1.34,0



Columns: ['subject_id', 'hadm_id', 'stay_id', 'gender', 'admission_age', 'race', 'icu_intime', 'icu_outtime', 'los_icu', 'hospital_expire_flag']
Shape: (5, 10)


In [18]:
# 3. Query specific materialized view (example: mv_icd9_patients)
mv_icd9_patients = pd.read_sql(
    "SELECT * FROM mimiciv_derived.mv_icd9_patients LIMIT 10;", 
    engine
)
mv_icd9_patients

,subject_id
0,18369563
1,15189519
2,14541250
3,17154054
4,17169170
5,19700833
6,13249543
7,10325137
8,15786954
9,14121296


In [21]:
# 4. View mv_icd9_icu_cohort_data (example with more columns)
cohort_data = pd.read_sql(
    """
    SELECT * 
    FROM mimiciv_derived.mv_icd9_icu_cohort_data;    """, 
    engine
)
cohort_data

,subject_id,hadm_id,stay_id,seq_num,icd_code,icd_version,long_title,admittime,dischtime,hospital_expire_flag
0,10016810,20973395,35436337,10,V4577,9,"Acquired absence of organ, genital organs",2185-06-16 01:31:00,2185-06-21 15:55:00,0
1,10016810,20973395,35436337,9,V1082,9,Personal history of malignant melanoma of skin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
2,10016810,20973395,35436337,8,V5866,9,Long-term (current) use of aspirin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
3,10016810,20973395,35436337,7,3051,9,Tobacco use disorder,2185-06-16 01:31:00,2185-06-21 15:55:00,0
4,10016810,20973395,35436337,6,25000,9,Diabetes mellitus without mention of complicat...,2185-06-16 01:31:00,2185-06-21 15:55:00,0
...,...,...,...,...,...,...,...,...,...,...
391712,19999287,22997012,37692584,5,496,9,"Chronic airway obstruction, not elsewhere clas...",2197-07-26 03:29:00,2197-07-31 14:00:00,0
391713,19999287,22997012,37692584,4,25000,9,Diabetes mellitus without mention of complicat...,2197-07-26 03:29:00,2197-07-31 14:00:00,0
391714,19999287,22997012,37692584,3,7907,9,Bacteremia,2197-07-26 03:29:00,2197-07-31 14:00:00,0
391715,19999287,22997012,37692584,2,486,9,"Pneumonia, organism unspecified",2197-07-26 03:29:00,2197-07-31 14:00:00,0


In [20]:
# 5. View mv_management_view_data
management_data = pd.read_sql(
    """
    SELECT * 
    FROM mimiciv_derived.mv_management_view_data 
    LIMIT 10;
    """, 
    engine
)
management_data

,subject_id,hadm_id,stay_id,gender,admission_age,race,icu_intime,icu_outtime,los_icu,hospital_expire_flag
0,10000032,29079034,39553978,F,52.559969,WHITE,2180-07-23 14:00:00,2180-07-23 23:50:47,0.41,0
1,10000690,25860671,37081114,F,86.837120,WHITE,2150-11-02 19:37:00,2150-11-06 17:03:17,3.89,0
2,10001217,24597018,37067082,F,55.881486,WHITE,2157-11-20 19:18:02,2157-11-21 22:08:00,1.12,0
3,10001217,27703517,34592300,F,55.962942,WHITE,2157-12-19 15:42:24,2157-12-20 14:27:41,0.95,0
4,10001725,25563031,31205490,F,46.275517,WHITE,2110-04-11 15:52:22,2110-04-12 23:59:56,1.34,0
5,10002155,20345487,32358465,F,83.186535,WHITE,2131-03-09 21:33:00,2131-03-10 18:09:21,0.86,1
6,10002155,23822395,33685454,F,81.592179,WHITE,2129-08-04 12:45:00,2129-08-10 17:02:38,6.18,0
7,10002155,28994087,31090461,F,82.729467,WHITE,2130-09-24 00:50:00,2130-09-27 22:13:41,3.89,0
8,10002760,28094813,31831386,M,56.299259,UNABLE TO OBTAIN,2141-04-20 13:20:46,2141-04-21 14:26:49,1.05,0
9,10003046,26048429,35514836,M,64.003565,WHITE,2154-01-02 15:57:15,2154-01-04 15:19:56,1.97,0


In [22]:
cohort_data

,subject_id,hadm_id,stay_id,seq_num,icd_code,icd_version,long_title,admittime,dischtime,hospital_expire_flag
0,10016810,20973395,35436337,10,V4577,9,"Acquired absence of organ, genital organs",2185-06-16 01:31:00,2185-06-21 15:55:00,0
1,10016810,20973395,35436337,9,V1082,9,Personal history of malignant melanoma of skin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
2,10016810,20973395,35436337,8,V5866,9,Long-term (current) use of aspirin,2185-06-16 01:31:00,2185-06-21 15:55:00,0
3,10016810,20973395,35436337,7,3051,9,Tobacco use disorder,2185-06-16 01:31:00,2185-06-21 15:55:00,0
4,10016810,20973395,35436337,6,25000,9,Diabetes mellitus without mention of complicat...,2185-06-16 01:31:00,2185-06-21 15:55:00,0
...,...,...,...,...,...,...,...,...,...,...
391712,19999287,22997012,37692584,5,496,9,"Chronic airway obstruction, not elsewhere clas...",2197-07-26 03:29:00,2197-07-31 14:00:00,0
391713,19999287,22997012,37692584,4,25000,9,Diabetes mellitus without mention of complicat...,2197-07-26 03:29:00,2197-07-31 14:00:00,0
391714,19999287,22997012,37692584,3,7907,9,Bacteremia,2197-07-26 03:29:00,2197-07-31 14:00:00,0
391715,19999287,22997012,37692584,2,486,9,"Pneumonia, organism unspecified",2197-07-26 03:29:00,2197-07-31 14:00:00,0


In [23]:
# Run Elixhauser Quan using mv_icd9_icu_cohort_data
print("Running Elixhauser Quan from cohort data...")
print("=" * 60)

# Read and execute the SQL script
with open('elixhauser_quan_from_cohort.sql', 'r') as f:
    sql_content = f.read()

try:
    with engine.begin() as conn:
        conn.execute(text(sql_content))
    print("✓ Successfully created public.elixhauser_quan table")
    
    # Check the results
    result_query = "SELECT COUNT(*) as row_count FROM public.elixhauser_quan;"
    count_df = pd.read_sql(result_query, engine)
    print(f"✓ Total rows in elixhauser_quan: {count_df['row_count'].iloc[0]:,}")
    
    # Show sample data
    sample_query = "SELECT * FROM public.elixhauser_quan LIMIT 5;"
    sample_df = pd.read_sql(sample_query, engine)
    print("\nFirst 5 rows:")
    display(sample_df)
    
except Exception as e:
    print(f"✗ Error: {str(e)[:500]}")

Running Elixhauser Quan from cohort data...
✓ Successfully created public.elixhauser_quan table
✓ Total rows in elixhauser_quan: 26,105

First 5 rows:


,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,20000808,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,20001361,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,20001976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20002270,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20002356,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# View the full elixhauser_quan table
elixhauser_df = pd.read_sql(
    "SELECT * FROM public.elixhauser_quan;", 
    engine
)
elixhauser_df

,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,20000808,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,20001361,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,20001976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20002270,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20002356,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26100,29998115,0,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
26101,29999098,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
26102,29999186,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26103,29999498,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [35]:
cols_keep = [
    "subject_id",
    "hadm_id",
    "stay_id",
    "admittime",
    "dischtime",
    "hospital_expire_flag",
]

admission_df = (
    cohort_data[cols_keep]
    .drop_duplicates(subset=["hadm_id"])
)


In [36]:
admission_df

,subject_id,hadm_id,stay_id,admittime,dischtime,hospital_expire_flag
0,10016810,20973395,35436337,2185-06-16 01:31:00,2185-06-21 15:55:00,0
10,10025612,23403708,32587226,2125-09-25 07:15:00,2125-10-03 12:24:00,0
26,10096969,25079335,36452771,2190-02-03 13:20:00,2190-02-05 17:00:00,0
33,10098993,21687208,35673013,2166-02-16 09:32:00,2166-02-21 14:55:00,0
55,10141035,24374681,38562441,2144-11-16 22:10:00,2144-11-22 16:12:00,0
...,...,...,...,...,...,...
391593,19970470,23848017,33533600,2146-12-05 15:56:00,2146-12-07 18:40:00,1
391615,19974380,27355109,35371457,2182-06-30 22:33:00,2182-07-12 18:10:00,0
391654,19984259,20109446,37233855,2153-11-29 23:43:00,2153-12-08 17:05:00,0
391678,19992202,20329411,36409124,2153-02-26 00:48:00,2153-03-02 17:00:00,0


In [37]:
final_df = admission_df.merge(
    elixhauser_df,
    on="hadm_id",
    how="left"
)


In [38]:
final_df

,subject_id,hadm_id,stay_id,admittime,dischtime,hospital_expire_flag,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,10016810,20973395,35436337,2185-06-16 01:31:00,2185-06-21 15:55:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10025612,23403708,32587226,2125-09-25 07:15:00,2125-10-03 12:24:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10096969,25079335,36452771,2190-02-03 13:20:00,2190-02-05 17:00:00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10098993,21687208,35673013,2166-02-16 09:32:00,2166-02-21 14:55:00,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10141035,24374681,38562441,2144-11-16 22:10:00,2144-11-22 16:12:00,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26100,19970470,23848017,33533600,2146-12-05 15:56:00,2146-12-07 18:40:00,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
26101,19974380,27355109,35371457,2182-06-30 22:33:00,2182-07-12 18:10:00,0,1,1,1,0,...,1,0,0,1,1,0,0,0,0,0
26102,19984259,20109446,37233855,2153-11-29 23:43:00,2153-12-08 17:05:00,0,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
26103,19992202,20329411,36409124,2153-02-26 00:48:00,2153-03-02 17:00:00,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [39]:
sepsis_df = pd.read_sql(
    "SELECT * FROM mimiciv_derived.sepsis3;", 
    engine
)

In [40]:
sepsis_df

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,18421337,30000484,2136-01-14 21:00:00,2136-01-14 18:10:00,2136-01-14 18:10:00,2136-01-14 19:00:00,3,0,0,0,0,3,0,True
1,12207593,30000646,2194-04-29 07:00:00,2194-04-29 01:00:00,2194-04-29 01:00:00,2194-04-29 11:00:00,3,2,0,0,1,0,0,True
2,15726459,30000831,2140-04-18 18:00:00,2140-04-18 05:11:00,2140-04-18 05:11:00,2140-04-17 22:00:00,5,0,0,0,0,3,2,True
3,16513856,30001446,2186-04-12 04:00:00,2186-04-11 08:20:00,2186-04-11 08:20:00,2186-04-12 04:00:00,8,0,3,3,0,0,2,True
4,10656173,30001555,2177-09-27 16:00:00,2177-09-27 07:21:00,2177-09-27 07:21:00,2177-09-27 12:00:00,8,0,3,4,0,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41290,15954569,39998871,2180-02-24 10:00:00,2180-02-24 04:05:00,2180-02-24 04:05:00,2180-02-24 08:00:00,3,0,0,0,3,0,0,True
41291,19425920,39999168,2190-04-11 21:00:00,2190-04-11 21:57:00,2190-04-11 21:00:00,2190-04-11 20:00:00,2,0,0,0,1,1,0,True
41292,15669140,39999172,2185-02-17 20:00:00,2185-02-18 12:45:00,2185-02-17 20:00:00,2185-02-17 18:00:00,2,0,0,0,1,0,1,True
41293,13651601,39999230,2147-09-01 02:00:00,2147-08-31 20:59:00,2147-08-31 20:59:00,2147-08-31 23:00:00,2,0,0,0,1,0,1,True


In [46]:
sepsis_comorbidity_df = final_df.merge(sepsis_df, on=["subject_id",'stay_id'], how="left")

In [49]:
sepsis_comorbidity_df

,subject_id,hadm_id,stay_id,admittime,dischtime,hospital_expire_flag,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,...,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,10016810,20973395,35436337,2185-06-16 01:31:00,2185-06-21 15:55:00,0,0,0,0,0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10025612,23403708,32587226,2125-09-25 07:15:00,2125-10-03 12:24:00,0,0,0,0,0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10096969,25079335,36452771,2190-02-03 13:20:00,2190-02-05 17:00:00,0,0,0,0,0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10098993,21687208,35673013,2166-02-16 09:32:00,2166-02-21 14:55:00,0,1,1,0,0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10141035,24374681,38562441,2144-11-16 22:10:00,2144-11-22 16:12:00,0,0,0,0,0,...,2144-11-16 18:30:00,2144-11-17 04:00:00,5.0,0.0,0.0,0.0,1.0,0.0,4.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26100,19970470,23848017,33533600,2146-12-05 15:56:00,2146-12-07 18:40:00,1,0,0,0,0,...,2146-12-05 17:00:00,2146-12-05 17:00:00,11.0,3.0,0.0,0.0,4.0,0.0,4.0,True
26101,19974380,27355109,35371457,2182-06-30 22:33:00,2182-07-12 18:10:00,0,1,1,1,0,...,2182-07-01 04:59:00,2182-07-01 10:00:00,6.0,0.0,1.0,0.0,0.0,2.0,3.0,True
26102,19984259,20109446,37233855,2153-11-29 23:43:00,2153-12-08 17:05:00,0,1,1,0,1,...,2153-11-30 20:10:00,2153-11-30 21:00:00,4.0,2.0,0.0,0.0,0.0,2.0,0.0,True
26103,19992202,20329411,36409124,2153-02-26 00:48:00,2153-03-02 17:00:00,0,0,0,0,0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
sepsis_comorbidity_df.to_csv("sepsis_comorbidity.csv", index=False)

In [50]:
sepsis_comorbidity_df['sepsis3'] = sepsis_comorbidity_df['sepsis3'].fillna(0)

In [52]:
sepsis_comorbidity_df['sepsis3'].value_counts()

sepsis3
0       13390
True    12715
Name: count, dtype: int64

In [78]:
# Extract only subject_id from sepsis_comorbidity_df
subject_ids = sepsis_comorbidity_df['subject_id']
subject_ids

0        10016810
1        10025612
2        10096969
3        10098993
4        10141035
           ...   
26100    19970470
26101    19974380
26102    19984259
26103    19992202
26104    19999287
Name: subject_id, Length: 26105, dtype: int64

In [ ]:
measurements = {
    "coagulation": {
        "d_dimer": [51196],
        "fibrinogen": [51214],
        "thrombin": [51297],
        "inr": [51237],
        "pt": [51274],
        "ptt": [51275],
    },

    "vital_signs": {
        "heart_rate": [220045],
        "sbp": [220179, 220050, 225309],
        "dbp": [220180, 220051, 225310],
        "mbp": [220052, 220181, 225312],
        "sbp_ni": [220179],
        "dbp_ni": [220180],
        "mbp_ni": [220181],
        "resp_rate": [220210, 224690],
        "temperature": [223761, 223762],
        "temperature_site": [224642],
        "spo2": [220277],
        "glucose": [225664, 220621, 226537],
    },

    "chemistry": {
        "creatinine": [50912],
    },

    "enzymes": {
        "bilirubin_total": [50885],
        "bilirubin_direct": [50883],
        "bilirubin_indirect": [50884],
    },

    "complete_blood_count": {
    "wbc": [51300],
    "neutrophils": [51256],
    "bands": [51144],
    "lymphocytes": [51244],
    "platelet": [51265],
    },

    "blood_gas": {
    "ph": [50820],
    "pco2": [50818],
    "po2": [50821],
    "base_excess": [50802],
    "bicarbonate": [50803],
    },
    
    # derived from outputevents, converted to rate downstream
    "urine_output": {
        "urine_output_volume": [
            226559,  # Foley
            226560,  # Void
            226561,  # Con dom Cath
            226584,  # Ileoconduit
            226563,  # Suprapubic
            226564,  # R Nephrostomy
            226565,  # L Nephrostomy
            226567,  # Straight Cath
            226557,  # R Ureteral Stent
            226558,  # L Ureteral Stent
            227488,  # GU Irrigant Volume In (negative)
            227489,  # GU Irrigant / Urine Volume Out
        ]
    },

    "treatments_vasopressors": {
        "epinephrine": [221289],
        "dopamine": [221662],
        "norepinephrine": [221906],
        "dobutamine": [221653],
    }
}

In [76]:
vital_signs = {
    "heart_rate": [220045],
    "sbp": [220179, 220050, 225309],
    "dbp": [220180, 220051, 225310],
    "mbp": [220052, 220181, 225312],
    "resp_rate": [220210, 224690],
    "temperature": [223761, 223762],
    "temperature_site": [224642],   # usually categorical; probably drop from numeric features
    "spo2": [220277],
    "glucose": [225664, 220621, 226537],
}

all_itemids = sorted({iid for ids in vital_signs.values() for iid in ids})


In [ ]:
# Get unique subject_ids from sepsis_comorbidity_df
subject_ids_list = sepsis_comorbidity_df['subject_id'].unique().tolist()
subject_ids_str = ",".join(map(str, subject_ids_list))
cohort = sepsis_comorbidity_df[["stay_id"]].drop_duplicates()
cohort.to_sql("tmp_cohort_stays", con=engine, index=False, if_exists="replace")
itemid_str = ",".join(map(str, all_itemids))
query = f"""
SELECT
  ce.subject_id,
  ce.hadm_id,
  ce.stay_id,
  ce.charttime,
  ce.itemid,
  ce.valuenum
FROM mimiciv_icu.chartevents ce
JOIN tmp_cohort_stays c
  ON ce.stay_id = c.stay_id
JOIN mimiciv_icu.icustays icu
  ON ce.stay_id = icu.stay_id
WHERE ce.valuenum IS NOT NULL
  AND ce.itemid IN ({itemid_str})
  AND ce.charttime >= icu.intime
  AND ce.charttime <= icu.outtime
ORDER BY ce.stay_id, ce.charttime;
"""
vitals_long = pd.read_sql(query, engine, parse_dates=["charttime"])

In [86]:
icu_times = pd.read_sql(
    "SELECT stay_id, intime, outtime FROM mimiciv_icu.icustays;",
    engine,
    parse_dates=["intime","outtime"]
)

vitals_long = vitals_long.merge(icu_times[["stay_id","intime"]], on="stay_id", how="left")

vitals_long["hour"] = (
    (vitals_long["charttime"] - vitals_long["intime"]).dt.total_seconds() // 3600
).astype("int64")

# keep only non-negative hours (safety)
vitals_long = vitals_long[vitals_long["hour"] >= 0]

In [87]:
vitals_long

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum,concept,intime,hour
0,12466550,23998182,30000153,2174-09-29 12:09:00,220210,18.0,resp_rate,2174-09-29 12:09:00,0
1,12466550,23998182,30000153,2174-09-29 13:00:00,220179,113.0,sbp,2174-09-29 12:09:00,0
2,12466550,23998182,30000153,2174-09-29 13:00:00,220180,77.0,dbp,2174-09-29 12:09:00,0
3,12466550,23998182,30000153,2174-09-29 13:00:00,220045,104.0,heart_rate,2174-09-29 12:09:00,0
4,12466550,23998182,30000153,2174-09-29 13:00:00,220277,100.0,spo2,2174-09-29 12:09:00,0
...,...,...,...,...,...,...,...,...,...
14677043,11305179,22341980,39999286,2158-03-08 14:00:00,220210,21.0,resp_rate,2158-03-07 11:15:18,26
14677044,11305179,22341980,39999286,2158-03-08 14:00:00,220277,94.0,spo2,2158-03-07 11:15:18,26
14677045,11305179,22341980,39999286,2158-03-08 14:11:00,220179,136.0,sbp,2158-03-07 11:15:18,26
14677046,11305179,22341980,39999286,2158-03-08 14:11:00,220181,94.0,mbp,2158-03-07 11:15:18,26


In [113]:
vitals_long["itemid"].unique()

array([220210, 220179, 220180, 220045, 220277, 220181, 220051, 220050,
       226537, 223761, 220052, 224690, 220621, 225664, 225309, 225312,
       225310, 223762])

In [95]:
vitals_long[vitals_long['subject_id']==12466550].head(30)

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum,concept,intime,hour
0,12466550,23998182,30000153,2174-09-29 12:09:00,220210,18.0,resp_rate,2174-09-29 12:09:00,0
1,12466550,23998182,30000153,2174-09-29 13:00:00,220179,113.0,sbp,2174-09-29 12:09:00,0
2,12466550,23998182,30000153,2174-09-29 13:00:00,220180,77.0,dbp,2174-09-29 12:09:00,0
3,12466550,23998182,30000153,2174-09-29 13:00:00,220045,104.0,heart_rate,2174-09-29 12:09:00,0
4,12466550,23998182,30000153,2174-09-29 13:00:00,220277,100.0,spo2,2174-09-29 12:09:00,0
5,12466550,23998182,30000153,2174-09-29 13:00:00,220210,16.0,resp_rate,2174-09-29 12:09:00,0
6,12466550,23998182,30000153,2174-09-29 13:00:00,220181,84.0,mbp,2174-09-29 12:09:00,0
7,12466550,23998182,30000153,2174-09-29 13:16:00,220051,72.0,dbp,2174-09-29 12:09:00,1
8,12466550,23998182,30000153,2174-09-29 13:16:00,220050,151.0,sbp,2174-09-29 12:09:00,1
9,12466550,23998182,30000153,2174-09-29 13:27:00,226537,158.0,glucose,2174-09-29 12:09:00,1


In [ ]:
vitals_long['subject_id']

subject_id
14411859    18380
14816494    14012
13697731    13696
10222191    13166
16126164    12129
            ...  
11897844        3
17889152        2
11803145        2
16790435        2
17047107        2
Name: count, Length: 26102, dtype: int64

In [108]:
vitals_long.to_csv("vitals_long.csv", index=False)

In [96]:
agg = (
    vitals_long
    .groupby(["stay_id", "hour", "concept"])["valuenum"]
    .agg(["mean", "min", "max", "last", "count"])
    .reset_index()
)

# rename columns
agg = agg.rename(columns={
    "mean":"val_mean",
    "min":"val_min",
    "max":"val_max",
    "last":"val_last",
    "count":"n_obs"
})


In [97]:
agg

,stay_id,hour,concept,val_mean,val_min,val_max,val_last,n_obs
0,30000153,0,dbp,77.0,77.0,77.0,77.0,1
1,30000153,0,heart_rate,104.0,104.0,104.0,104.0,1
2,30000153,0,mbp,84.0,84.0,84.0,84.0,1
3,30000153,0,resp_rate,17.0,16.0,18.0,16.0,2
4,30000153,0,sbp,113.0,113.0,113.0,113.0,1
...,...,...,...,...,...,...,...,...
12435637,39999286,26,heart_rate,95.0,95.0,95.0,95.0,1
12435638,39999286,26,mbp,94.0,94.0,94.0,94.0,1
12435639,39999286,26,resp_rate,21.0,21.0,21.0,21.0,1
12435640,39999286,26,sbp,136.0,136.0,136.0,136.0,1


In [98]:
wide = agg.pivot_table(
    index=["stay_id","hour"],
    columns="concept",
    values=["val_mean","val_min","val_max","val_last","n_obs"]
)

wide.columns = [f"{stat}_{concept}" for stat, concept in wide.columns]
wide = wide.reset_index()


In [99]:
for concept in vital_signs.keys():
    col = f"val_last_{concept}"
    if col in wide.columns:
        wide[f"missing_{concept}"] = wide[col].isna().astype(int)


In [100]:
wide

,stay_id,hour,n_obs_dbp,n_obs_glucose,n_obs_heart_rate,n_obs_mbp,n_obs_resp_rate,n_obs_sbp,n_obs_spo2,n_obs_temperature,...,val_min_spo2,val_min_temperature,missing_heart_rate,missing_sbp,missing_dbp,missing_mbp,missing_resp_rate,missing_temperature,missing_spo2,missing_glucose
0,30000153,0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,NaN,...,100.0,NaN,0,0,0,0,0,1,0,1
1,30000153,1,2.0,2.0,NaN,1.0,1.0,2.0,NaN,1.0,...,NaN,99.1,1,0,0,0,0,0,1,0
2,30000153,2,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,...,100.0,NaN,0,1,1,1,1,1,0,1
3,30000153,3,2.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,...,100.0,99.5,0,0,0,0,0,0,0,0
4,30000153,4,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,100.0,NaN,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976960,39999286,22,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,95.0,NaN,0,0,0,0,0,1,0,0
1976961,39999286,23,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,95.0,NaN,0,0,0,0,0,1,0,1
1976962,39999286,24,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,96.0,97.9,0,0,0,0,0,0,0,1
1976963,39999286,25,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,93.0,NaN,0,0,0,0,0,1,0,1


In [101]:
wide['stay_id'].nunique()

26102

In [102]:
wide = wide.sort_values(["stay_id","hour"])

for concept in vital_signs.keys():
    col = f"val_last_{concept}"
    if col not in wide.columns:
        continue

    observed = wide[col].notna().astype(int)

    # last observed hour index (forward fill)
    last_obs_hour = (
        wide["hour"].where(observed == 1)
        .groupby(wide["stay_id"])
        .ffill()
    )

    wide[f"delta_t_{concept}"] = wide["hour"] - last_obs_hour
    # if never observed yet, keep big number or fill with hour+1
    wide[f"delta_t_{concept}"] = wide[f"delta_t_{concept}"].fillna(wide["hour"] + 1)


In [107]:
wide[wide['stay_id']==30000153]

,stay_id,hour,n_obs_dbp,n_obs_glucose,n_obs_heart_rate,n_obs_mbp,n_obs_resp_rate,n_obs_sbp,n_obs_spo2,n_obs_temperature,...,missing_spo2,missing_glucose,delta_t_heart_rate,delta_t_sbp,delta_t_dbp,delta_t_mbp,delta_t_resp_rate,delta_t_temperature,delta_t_spo2,delta_t_glucose
0,30000153,0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,30000153,1,2.0,2.0,NaN,1.0,1.0,2.0,NaN,1.0,...,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,30000153,2,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,...,0,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,30000153,3,2.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30000153,4,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
5,30000153,5,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
6,30000153,6,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0
7,30000153,7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,30000153,8,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9,30000153,9,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,0,1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [111]:
vitals_long['concept'].value_counts()


concept
resp_rate      2377702
mbp            2298330
sbp            2293824
dbp            2293107
heart_rate     2210296
spo2           2163966
temperature     597614
glucose         442209
Name: count, dtype: int64

In [1]:
import pandas as pd
df = pd.read_csv("/Users/gloriaye/Downloads/DesktopOrganizer/Main/dsc180b/vitals_long.csv")

In [2]:
df

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum,concept,intime,hour
0,12466550,23998182,30000153,2174-09-29 12:09:00,220210,18.0,resp_rate,2174-09-29 12:09:00,0
1,12466550,23998182,30000153,2174-09-29 13:00:00,220179,113.0,sbp,2174-09-29 12:09:00,0
2,12466550,23998182,30000153,2174-09-29 13:00:00,220180,77.0,dbp,2174-09-29 12:09:00,0
3,12466550,23998182,30000153,2174-09-29 13:00:00,220045,104.0,heart_rate,2174-09-29 12:09:00,0
4,12466550,23998182,30000153,2174-09-29 13:00:00,220277,100.0,spo2,2174-09-29 12:09:00,0
...,...,...,...,...,...,...,...,...,...
14677043,11305179,22341980,39999286,2158-03-08 14:00:00,220210,21.0,resp_rate,2158-03-07 11:15:18,26
14677044,11305179,22341980,39999286,2158-03-08 14:00:00,220277,94.0,spo2,2158-03-07 11:15:18,26
14677045,11305179,22341980,39999286,2158-03-08 14:11:00,220179,136.0,sbp,2158-03-07 11:15:18,26
14677046,11305179,22341980,39999286,2158-03-08 14:11:00,220181,94.0,mbp,2158-03-07 11:15:18,26


In [4]:
df[df['itemid'] ==220179]

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum,concept,intime,hour
1,12466550,23998182,30000153,2174-09-29 13:00:00,220179,113.0,sbp,2174-09-29 12:09:00,0
126,12466550,23998182,30000153,2174-09-30 05:43:00,220179,153.0,sbp,2174-09-29 12:09:00,17
132,12466550,23998182,30000153,2174-09-30 06:00:00,220179,140.0,sbp,2174-09-29 12:09:00,17
140,12466550,23998182,30000153,2174-09-30 07:00:00,220179,142.0,sbp,2174-09-29 12:09:00,18
150,12466550,23998182,30000153,2174-09-30 08:00:00,220179,158.0,sbp,2174-09-29 12:09:00,19
...,...,...,...,...,...,...,...,...,...
14677021,11305179,22341980,39999286,2158-03-08 10:00:00,220179,121.0,sbp,2158-03-07 11:15:18,22
14677027,11305179,22341980,39999286,2158-03-08 11:00:00,220179,132.0,sbp,2158-03-07 11:15:18,23
14677032,11305179,22341980,39999286,2158-03-08 12:00:00,220179,128.0,sbp,2158-03-07 11:15:18,24
14677040,11305179,22341980,39999286,2158-03-08 13:00:00,220179,137.0,sbp,2158-03-07 11:15:18,25
